This notebook was original ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

In [1]:
%cd /code

/code


---

The `load_dotenv()` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [2]:
from dotenv import load_dotenv
import textwrap

load_dotenv()

def wprint(string: str, max_width: int = 80) -> None:
    """Print `string` with a maximum widgth."""
    wrapped_string = textwrap.fill(string, max_width)
    print(wrapped_string)

# OpenAI Chat

Here's a simple example using `GPT-3.5` chat:

In [3]:
from llm_workflow.openai import OpenAIChat

model = OpenAIChat(model_name='gpt-3.5-turbo')
response = model("What is the meaning of life?")
wprint(response)

The meaning of life is a philosophical question that has been debated for
centuries. Different people and cultures have different beliefs and
interpretations. Some believe that the meaning of life is to seek happiness and
fulfillment, while others find meaning in religious or spiritual beliefs.
Ultimately, the meaning of life may be subjective and can vary from person to
person. It is up to each individual to explore and find their own sense of
purpose and meaning in life.


---

## Streaming

We can stream the response by providing a callback. The callback takes a single parameter of type `StreamingEvent` which has a `response` property containing each token streamed. In the example below, we simply print the response and end the printed message with the `|` character.

In [4]:
from llm_workflow.openai import OpenAIChat

model = OpenAIChat(
    model_name='gpt-3.5-turbo',
    streaming_callback=lambda x: print(x.response, end='|')
)
response = model("What is the meaning of life?")

The| meaning| of| life| is| a| philosophical| question| that| has| been| debated| for| centuries|.| Different| people| and| cultures| have| different| beliefs| and| interpretations|.| Some| believe| that| the| meaning| of| life| is| to| seek| happiness| and| fulfillment|,| while| others| find| meaning| in| religious| or| spiritual| beliefs|.| Ultimately|,| the| meaning| of| life| may| be| subjective| and| can| vary| from| person| to| person|.| It| is| up| to| each| individual| to| explore| and| find| their| own| sense| of| purpose| and| meaning| in| life|.|

Like the previous example, the full text is returned at the end:

In [5]:
wprint(response)

The meaning of life is a philosophical question that has been debated for
centuries. Different people and cultures have different beliefs and
interpretations. Some believe that the meaning of life is to seek happiness and
fulfillment, while others find meaning in religious or spiritual beliefs.
Ultimately, the meaning of life may be subjective and can vary from person to
person. It is up to each individual to explore and find their own sense of
purpose and meaning in life.


---

## Usage & Cost

We have access to the usage and costs through various properties below.

In [6]:
from llm_workflow.openai import OpenAIChat

model = OpenAIChat(model_name='gpt-3.5-turbo')
model("What is the capital of France?")

'The capital of France is Paris.'

In [7]:
print(f"Total Cost:            ${model.cost:.5f}")
print(f"Total Tokens:          {model.total_tokens:,}")
print(f"Total Prompt Tokens:   {model.input_tokens:,}")
print(f"Total Response Tokens: {model.response_tokens:,}")

Total Cost:            $0.00005
Total Tokens:          31
Total Prompt Tokens:   24
Total Response Tokens: 7


If we use the same model/object again, the cost/usage will be incremented accordingly.

In [8]:
model("What is the capital of Germany?")

'The capital of Germany is Berlin.'

In [9]:
print(f"Total Cost:            ${model.cost:.5f}")
print(f"Total Tokens:          {model.total_tokens:,}")
print(f"Total Prompt Tokens:   {model.input_tokens:,}")
print(f"Total Response Tokens: {model.response_tokens:,}")

Total Cost:            $0.00013
Total Tokens:          84
Total Prompt Tokens:   70
Total Response Tokens: 14


## History

We can use the `history()` method to get the prompt/response and cost/usage for each of the messages used by the model/object. 

There are two `ExchangeRecord` items in the list. The first item corresponds to the first question and the second item corresponds to the second question.

In [10]:
model.history()

[ExchangeRecord(uuid='bc91d946-b89c-4a19-80df-ce1b978e5cd9', timestamp='2023-10-16 00:18:16.774', metadata={'model_name': 'gpt-3.5-turbo', 'temperature': 0, 'max_tokens': 2000, 'timeout': 10, 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'What is the capital of France?'}]}, cost=5e-05, total_tokens=31, prompt='What is the capital of France?', response='The capital of France is Paris.', input_tokens=24, response_tokens=7),
 ExchangeRecord(uuid='47f9b0ed-93f0-49e1-8de9-afc2211f3e44', timestamp='2023-10-16 00:18:17.115', metadata={'model_name': 'gpt-3.5-turbo', 'temperature': 0, 'max_tokens': 2000, 'timeout': 10, 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'What is the capital of France?'}, {'role': 'assistant', 'content': 'The capital of France is Paris.'}, {'role': 'user', 'content': 'What is the capital of Germany?'}]}, cost=8.3e-05, total_tokens=53, prompt='What is the

In [11]:
print(f"prompt: {model.history()[0].prompt}")
print(f"Total Cost:            ${model.history()[0].cost:.5f}")
print(f"Total Tokens:          {model.history()[0].total_tokens:,}")
print(f"Total Prompt Tokens:   {model.history()[0].input_tokens:,}")
print(f"Total Response Tokens: {model.history()[0].response_tokens:,}")

prompt: What is the capital of France?
Total Cost:            $0.00005
Total Tokens:          31
Total Prompt Tokens:   24
Total Response Tokens: 7


In [12]:
print(f"prompt: {model.history()[1].prompt}")
print(f"Total Cost:            ${model.history()[1].cost:.5f}")
print(f"Total Tokens:          {model.history()[1].total_tokens:,}")
print(f"Total Prompt Tokens:   {model.history()[1].input_tokens:,}")
print(f"Total Response Tokens: {model.history()[1].response_tokens:,}")

prompt: What is the capital of Germany?
Total Cost:            $0.00008
Total Tokens:          53
Total Prompt Tokens:   46
Total Response Tokens: 7


---

In [13]:
print(len("What is the capital of France?"))
print(len("What is the capital of Germany?"))

30
31


In [14]:
print(f"Total Prompt Tokens (1st message):   {model.history()[0].input_tokens:,}")
print(f"Total Prompt Tokens (2nd message):   {model.history()[1].input_tokens:,}")

Total Prompt Tokens (1st message):   24
Total Prompt Tokens (2nd message):   46


Notice above that even though the two questions we've ask are very close in size, the number of `input_tokens` has almost doubled in the second question/message. This is because we are sending the entire list of messages to the model for each new question so the model has the context of the entire conversation. (see the [memory.ipynb](https://github.com/shane-kercheval/llm-workflow/tree/main/examples/memory.ipynb) notebook for examples of limiting the memory).

We can see exactly what we sent the model using the `_previous_messages` property below.

In [15]:
model._previous_messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'What is the capital of France?'},
 {'role': 'assistant', 'content': 'The capital of France is Paris.'},
 {'role': 'user', 'content': 'What is the capital of Germany?'}]

---